In [4]:
#逆向最大匹配
class IMM(object):
    def __init__(self, dic_path):
        self.dictionary = set()
        self.maximum = 0
        #读取词典
        with open(dic_path, 'r', encoding='utf8') as f:
            for line in f:
                line = line.strip()
                if not line:
                    continue
                self.dictionary.add(line)
                if len(line) > self.maximum:
                    self.maximum = len(line)
    def cut(self, text):
        result = []
        index = len(text)
        while index > 0:
            word = None
            for size in range(self.maximum, 0, -1):
                if index - size < 0:
                    continue
                piece = text[(index - size):index]
                if piece in self.dictionary:
                    word = piece
                    result.append(word)
                    index -= size
                    break
            if word is None:
                index -= 1
        return result[::-1]

def main():
    text = "南京市长江大桥"
    
    tokenizer = IMM('./data/imm_dic.utf8')
    print(tokenizer.cut(text))

main()

['南京市', '长江大桥']


In [3]:
class HMM(object):
    def __init__(self):
        pass
    
    def try_load_model(self, trained):
        pass
    
    def train(self, path):
        pass
    
    def viterbi(self, text, states, start_p, trans_p, emit_p):
        pass
    
    def cut(self, text):
        pass

In [138]:
class HMM(object):
    def __init__(self):
        import os

        # 主要是用于存取算法中间结果，不用每次都训练模型
        self.model_file = './data/hmm_model.pkl'

        # 状态值集合
        self.state_list = ['B', 'M', 'E', 'S']
        # 参数加载,用于判断是否需要重新加载model_file
        self.load_para = False

    # 用于加载已计算的中间结果，当需要重新训练时，需初始化清空结果
    def try_load_model(self, trained):
        if trained:
            import pickle
            with open(self.model_file, 'rb') as f:
                self.A_dic = pickle.load(f)
                self.B_dic = pickle.load(f)
                self.Pi_dic = pickle.load(f)
                self.load_para = True

        else:
            # 状态转移概率（状态->状态的条件概率）
            self.A_dic = {}
            # 发射概率（状态->词语的条件概率）
            self.B_dic = {}
            # 状态的初始概率
            self.Pi_dic = {}
            self.load_para = False

    # 计算转移概率、发射概率以及初始概率
    def train(self, path):

        # 重置几个概率矩阵
        self.try_load_model(False)

        # 统计状态出现次数，求p(o)
        Count_dic = {}

        # 初始化参数
        def init_parameters():
            for state in self.state_list:
                self.A_dic[state] = {s: 0.0 for s in self.state_list}
                self.Pi_dic[state] = 0.0
                self.B_dic[state] = {}

                Count_dic[state] = 0

        def makeLabel(text):
            out_text = []
            if len(text) == 1:
                out_text.append('S')
            else:
                out_text += ['B'] + ['M'] * (len(text) - 2) + ['E']

            return out_text

        init_parameters()
        line_num = -1
        # 观察者集合，主要是字以及标点等
        words = set()
        with open(path, encoding='utf8') as f:
            for line in f:
                line_num += 1

                line = line.strip()
                if not line:
                    continue

                word_list = [i for i in line if i != ' ']
                words |= set(word_list)  # 更新字的集合

                linelist = line.split()

                line_state = []
                for w in linelist:
                    line_state.extend(makeLabel(w))
                
                assert len(word_list) == len(line_state)

                for k, v in enumerate(line_state):
                    Count_dic[v] += 1
                    if k == 0:
                        self.Pi_dic[v] += 1  # 每个句子的第一个字的状态，用于计算初始状态概率
                    else:
                        self.A_dic[line_state[k - 1]][v] += 1  # 计算转移概率
                        self.B_dic[line_state[k]][word_list[k]] = \
                            self.B_dic[line_state[k]].get(word_list[k], 0) + 1.0  # 计算发射概率
        
        self.Pi_dic = {k: v * 1.0 / line_num for k, v in self.Pi_dic.items()}
        self.A_dic = {k: {k1: v1 / Count_dic[k] for k1, v1 in v.items()}
                      for k, v in self.A_dic.items()}
        #加1平滑
        self.B_dic = {k: {k1: (v1 + 1) / Count_dic[k] for k1, v1 in v.items()}
                      for k, v in self.B_dic.items()}
        #序列化
        import pickle
        with open(self.model_file, 'wb') as f:
            pickle.dump(self.A_dic, f)
            pickle.dump(self.B_dic, f)
            pickle.dump(self.Pi_dic, f)

        return self

    def viterbi(self, text, states, start_p, trans_p, emit_p):
        V = [{}]
        path = {}
        for y in states:
            V[0][y] = start_p[y] * emit_p[y].get(text[0], 0)
            path[y] = [y]
        for t in range(1, len(text)):
            V.append({})
            newpath = {}
            
            #检验训练的发射概率矩阵中是否有该字
            neverSeen = text[t] not in emit_p['S'].keys() and \
                text[t] not in emit_p['M'].keys() and \
                text[t] not in emit_p['E'].keys() and \
                text[t] not in emit_p['B'].keys()
            for y in states:
                emitP = emit_p[y].get(text[t], 0) if not neverSeen else 1.0 #设置未知字单独成词
                (prob, state) = max(
                    [(V[t - 1][y0] * trans_p[y0].get(y, 0) *
                      emitP, y0)
                     for y0 in states if V[t - 1][y0] > 0])
                V[t][y] = prob
                newpath[y] = path[state] + [y]
            path = newpath
            
        if emit_p['M'].get(text[-1], 0)> emit_p['S'].get(text[-1], 0):
            (prob, state) = max([(V[len(text) - 1][y], y) for y in ('E','M')])
        else:
            (prob, state) = max([(V[len(text) - 1][y], y) for y in states])
        
        return (prob, path[state])

    def cut(self, text):
        import os
        if not self.load_para:
            self.try_load_model(os.path.exists(self.model_file))
        prob, pos_list = self.viterbi(text, self.state_list, self.Pi_dic, self.A_dic, self.B_dic)      
        begin, next = 0, 0    
        for i, char in enumerate(text):
            pos = pos_list[i]
            if pos == 'B':
                begin = i
            elif pos == 'E':
                yield text[begin: i+1]
                next = i+1
            elif pos == 'S':
                yield char
                next = i+1
        if next < len(text):
            yield text[next:]



In [142]:
hmm = HMM()
hmm.train('./data/trainCorpus.txt_utf8')

text = '这是一个非常棒的方案！'
res = hmm.cut(text)
print(text)
print(str(list(res)))

这是一个非常棒的方案！
['这是', '一个', '非常', '棒', '的', '方案', '！']


In [27]:
#jieba分词示例
def get_content(path):
    
    with open(path, 'r', encoding='gbk', errors='ignore') as f:
        content = ''
        for l in f:
            l = l.strip()
            content += l
        return content


def get_TF(words, topK=10):
    
    tf_dic = {}
    for w in words:
        tf_dic[w] = tf_dic.get(w, 0) + 1
    return sorted(tf_dic.items(), key = lambda x: x[1], reverse=True)[:topK]

def stop_words(path):
    with open(path) as f:
        return [l.strip() for l in f]
stop_words('./data/stop_words.utf8')

#分词
def main():
    import glob
    import random
    import jieba
    
    files = glob.glob('./data/news/C000013/*.txt')
    corpus = [get_content(x) for x in files[:5]]
    
    
    sample_inx = random.randint(0, len(corpus))
    sample_inx = 3
    
    import jieba.posseg as psg
    
    split_words = [x for x in jieba.cut(corpus[sample_inx]) if x not in stop_words('./data/stop_words.utf8')]
    print('样本之一：'+corpus[sample_inx])
    print('样本分词效果：'+'/ '.join(split_words))
    print('样本的topK（10）词：'+str(get_TF(split_words)))
main()

Building prefix dict from the default dictionary ...
Dumping model to file cache /var/folders/5d/hz_f9tvs0t37skhfcqsbspw00000gn/T/jieba.cache
Loading model cost 1.174 seconds.
Prefix dict has been built succesfully.


样本之一：中国卫生部官员24日说，截至2005年底，中国各地报告的尘肺病病人累计已超过60万例，职业病整体防治形势严峻。卫生部副部长陈啸宏在当日举行的“国家职业卫生示范企业授牌暨企业职业卫生交流大会”上说，中国各类急性职业中毒事故每年发生200多起，上千人中毒，直接经济损失达上百亿元。职业病病人总量大、发病率较高、经济损失大、影响恶劣。卫生部24日公布，2005年卫生部共收到全国30个省、自治区、直辖市（不包括西藏、港、澳、台）各类职业病报告12212例，其中尘肺病病例报告9173例，占75．11%。陈啸宏说，矽肺和煤工尘肺是中国最主要的尘肺病，且尘肺病发病工龄在缩短。去年报告的尘肺病病人中最短接尘时间不足三个月，平均发病年龄40．9岁，最小发病年龄20岁。陈啸宏表示，政府部门执法不严、监督不力，企业生产水平不高、技术设备落后等是职业卫生问题严重的原因。“但更重要的原因是有些企业法制观念淡薄，社会责任严重缺位，缺乏维护职工健康的强烈的意识，职工的合法权益不能得到有效的保障。”他说。为提高企业对职业卫生工作的重视，卫生部、国家安全生产监督管理总局和中华全国总工会24日在京评选出56家国家级职业卫生工作示范企业，希望这些企业为社会推广职业病防治经验，促使其他企业作好职业卫生工作，保护劳动者健康。
样本分词效果：中国卫生部/ 官员/ 24/ 日/ 2005/ 年底/ 中国/ 各地/ 报告/ 尘肺病/ 病人/ 累计/ 超过/ 60/ 万例/ 职业病/ 整体/ 防治/ 形势严峻/ 卫生部/ 副/ 部长/ 陈啸宏/ 当日/ 举行/ 国家/ 职业/ 卫生/ 示范/ 企业/ 授牌/ 暨/ 企业/ 职业/ 卫生/ 交流/ 大会/ 中国/ 各类/ 急性/ 职业/ 中毒/ 事故/ 每年/ 发生/ 200/ 多起/ 上千人/ 中毒/ 直接/ 经济损失/ 达上/ 百亿元/ 职业病/ 病人/ 总量/ 发病率/ 高/ 经济损失/ 影响/ 恶劣/ 卫生部/ 24/ 日/ 公布/ 2005/ 年/ 卫生部/ 共/ 收到/ 全国/ 30/ 个省/ 自治区/ 直辖市/ 包括/ 西藏/ 港/ 澳/ 台/ 各类/ 职业病/ 报告/ 12212/ 例/ 尘肺病/ 病例/ 报告/ 9173/ 例/ 占/ 75/ ．/ 11/ 陈啸宏/ 矽肺/ 煤工/ 尘肺/ 中国/ 主要/ 尘肺病/ 尘肺病/ 发病/ 工龄/ 

In [8]:
#jieba词性标注示例

def main():
    import glob
    import random
    import jieba
    import jieba.posseg as psg
    
    files = glob.glob('./data/news/C000013/*.txt')
    corpus = [get_content(x) for x in files]
    
    sample_inx = random.randint(0, len(corpus))
    sample_inx = 3
    
    split_words = [w for w, t in psg.cut(corpus[sample_inx]) 
                   if w not in stop_words('./data/stop_words.utf8')
                  and t.startswith('n')]
    print('样本之一：'+corpus[sample_inx])
    print('样本分词效果：'+'/ '.join(split_words))
    print('样本的topK（10）词：'+str(get_TF(split_words)))
main()

样本之一：中国卫生部官员24日说，截至2005年底，中国各地报告的尘肺病病人累计已超过60万例，职业病整体防治形势严峻。卫生部副部长陈啸宏在当日举行的“国家职业卫生示范企业授牌暨企业职业卫生交流大会”上说，中国各类急性职业中毒事故每年发生200多起，上千人中毒，直接经济损失达上百亿元。职业病病人总量大、发病率较高、经济损失大、影响恶劣。卫生部24日公布，2005年卫生部共收到全国30个省、自治区、直辖市（不包括西藏、港、澳、台）各类职业病报告12212例，其中尘肺病病例报告9173例，占75．11%。陈啸宏说，矽肺和煤工尘肺是中国最主要的尘肺病，且尘肺病发病工龄在缩短。去年报告的尘肺病病人中最短接尘时间不足三个月，平均发病年龄40．9岁，最小发病年龄20岁。陈啸宏表示，政府部门执法不严、监督不力，企业生产水平不高、技术设备落后等是职业卫生问题严重的原因。“但更重要的原因是有些企业法制观念淡薄，社会责任严重缺位，缺乏维护职工健康的强烈的意识，职工的合法权益不能得到有效的保障。”他说。为提高企业对职业卫生工作的重视，卫生部、国家安全生产监督管理总局和中华全国总工会24日在京评选出56家国家级职业卫生工作示范企业，希望这些企业为社会推广职业病防治经验，促使其他企业作好职业卫生工作，保护劳动者健康。
样本分词效果：中国卫生部/ 官员/ 中国/ 报告/ 尘肺病/ 病人/ 职业病/ 整体/ 卫生部/ 部长/ 陈啸宏/ 国家/ 职业/ 企业/ 暨/ 企业/ 职业/ 交流/ 大会/ 中国/ 急性/ 职业/ 中毒/ 事故/ 中毒/ 经济损失/ 职业病/ 病人/ 总量/ 发病率/ 经济损失/ 卫生部/ 卫生部/ 全国/ 省/ 自治区/ 直辖市/ 西藏/ 澳/ 职业病/ 报告/ 尘肺病/ 病例/ 报告/ 陈啸宏/ 矽肺/ 煤工/ 尘肺/ 中国/ 尘肺病/ 尘肺病/ 工龄/ 报告/ 尘肺病/ 病人/ 尘/ 时间/ 年龄/ 年龄/ 陈啸宏/ 政府部门/ 不力/ 企业/ 水平/ 技术设备/ 职业/ 问题/ 原因/ 原因/ 企业/ 法制观念/ 社会/ 责任/ 缺位/ 职工/ 意识/ 职工/ 合法权益/ 企业/ 职业/ 卫生部/ 国家/ 监督管理/ 总局/ 中华全国总工会/ 京/ 职业/ 企业/ 企业/ 社会/ 职业病/ 经验/ 企业/ 职业/ 劳动者
样本的topK（10）词：[('企业', 8